In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re
import os
import urllib 
import nltk

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *

#https://stackoverflow.com/questions/37513355/converting-pandas-dataframe-into-spark-dataframe-error


%matplotlib inline


In [2]:

# Getting the current directory of interest
thisdir = '/project/ds5559/twitter_sentiment_analysis_group/'

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Python Spark SQL basic example") \
        .config("spark.executor.memory", '20g') \
        .config('spark.executor.cores', '8') \
        .config('spark.executor.instances', '2') \
        .config("spark.driver.memory",'1g') \
        .appName("twitter_project") \
        .getOrCreate()

sc = spark.sparkContext

sqlContext = SQLContext(sc)

        # use all cores on local machine
    # will see appName on cluster manager
    # RAM per executor (worker)
    # cores available to EACH executor
#      total number of executors
# RAM for driver, generally lower need than a worker

#         .config("spark.driver.extraClassPath", "lib/sparknlp.jar") \
#         .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.0") \
    
# df = spark.read.csv(thisdir)

# df.take(5)

In [3]:


# Getting the current directory of interest
thisdir = '/project/ds5559/twitter_sentiment_analysis_group/'

def pd_df_from_csvs_in_directory(directory):
    """
    Accept directory, convert each CSV to pandas dataframe
    then join all dataframes to single dataframe output.
    """
    df = pd.DataFrame()
    # r=root, d=directories, f = files
    for r, d, f in os.walk(directory):
        for file in f:
            if file.endswith(".csv"):
                filedir = os.path.join(r, file)
                print(filedir)
                try:
                    df = pd.concat([df, pd.read_csv(filedir, index_col = 0)])
                except:
                    pass
                
    return df

dfALL = pd_df_from_csvs_in_directory(thisdir)


/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210318_0309.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210322_0254.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210320_1148.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210321_1119.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210318_1359.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210322_1518.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_early_attempt.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210319_1252.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210319_2253.csv
/project/ds5559/twitter_sentiment_analysis_group/testing_deleteme.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210320_0902.csv


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0,4,6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210323_0513.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210317_1203.csv
/project/ds5559/twitter_sentiment_analysis_group/tmp_pandas_df.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210329_0858.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210321_0531.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210321_0055.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210321_1044.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210327_0836.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210318_1401.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210320_4511.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210322_2104.csv
/project/ds5559/twitter_sentiment_analysis_group/hashtag_output_210323_1639.csv
/project/ds5559/twitter_sentiment_analysis_group/hash

In [4]:
len(dfALL.index)

2119921

In [5]:
%time

dfALL = dfALL.replace(r"[\n|\t|\r]", " ", regex=True)
dfALL.to_csv(thisdir + "tmp_pandas_df.csv", sep="\t")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.91 µs


In [6]:
df = spark.read.option("delimiter", "\t").option("header", "true").csv(thisdir + "tmp_pandas_df.csv")
df.count()

2119921

In [7]:
type(df)

pyspark.sql.dataframe.DataFrame

In [8]:
df.take(4)

[Row(_c0='0', scraped_hashtag='#vaccine', scraped_order='0.0', created_at='Thu Mar 18 01:05:26 +0000 2021', id_str='1.3723534175075656e+18', text='BIG NEWS from @GovSisolak: #COVID19Vaccine eligibility expands to all Nevadans aged 16+ w/ underlying conditions on… https://t.co/VNaabOLERs', truncated='True', in_reply_to_screen_name=None, retweet_count='0.0', favorite_count='0.0', lang='en', screen_name='atdleft', user_name='Andrew Davey', user_description="Muckraker, troublemaker, & no-BS-taker. I'm a Nevada based writer who's done a lot, but isn't done with it just yet.", user_verified='False', user_followers_count='2034.0', hashtags="['COVID19Vaccine']", symbols='[]', og_tweet_by=None, og_tweet_truncated=None),
 Row(_c0='1', scraped_hashtag='#vaccine', scraped_order='0.0', created_at='Thu Mar 18 01:05:15 +0000 2021', id_str='1.372353373123408e+18', text='@TerryBrady2097 This #Health #COVID19 #Vaccine rollout is sadly looking to be a repeat of the #agedcare pathway man… https://t.co/Rpo

In [9]:
#https://towardsdatascience.com/natural-language-processing-with-pyspark-and-spark-nlp-b5b29f8faba

!pip install sparknlp

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import sparknlp
sparknlp.start()

In [11]:
!java -version

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [12]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_242
Branch HEAD
Compiled by user centos on 2020-02-02T19:38:06Z
Revision cee4ecbb16917fa85f02c635925e2687400aa56b
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


In [13]:
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

In [14]:
# from sparknlp.base import Finisher, DocumentAssembler
# from pyspark.ml import Pipeline
# from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner

In [15]:
# documentAssembler = DocumentAssembler() \
#      .setInputCol('text') \
#      .setOutputCol('document')

# tokenizer = Tokenizer() \
#      .setInputCols(['document']) \
#      .setOutputCol('token')

# # note normalizer defaults to changing all words to lowercase.
# # Use .setLowercase(False) to maintain input case.""
# normalizer = Normalizer() \
#      .setInputCols(['token']) \
#      .setOutputCol('normalized') \
#      .setLowercase(True)

# # note that lemmatizer needs a dictionary. So I used the pre-trained
# # model (note that it defaults to english)
# lemmatizer = LemmatizerModel.pretrained() \
#      .setInputCols(['normalized']) \
#      .setOutputCol('lemma')

# stopwords_cleaner = StopWordsCleaner() \
#      .setInputCols(['lemma']) \
#      .setOutputCol('clean_lemma') \
#      .setCaseSensitive(False) \
#      .setStopWords(eng_stopwords)

# # finisher converts tokens to human-readable output
# finisher = Finisher() \
#      .setInputCols(['clean_lemma']) \
#      .setCleanAnnotations(False)

# pipeline = Pipeline() \
#      .setStages([
#            documentAssembler,
#            tokenizer,
#            normalizer,
#            lemmatizer,
#            stopwords_cleaner,
#            finisher
#      ])

In [16]:
# # transform text with the pipeline
# equifax = pipeline.fit(data).transform(data)

In [17]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/cam7cu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
dfdata = df.na.drop(subset=["text"]).drop()

In [19]:
df2 = dfdata.rdd\
    .map(lambda x: list(sid.polarity_scores(x['text']).values()))\
    .toDF(["neg", "neu", "pos", "compound"])
df2.show(5)

+-----+-----+-----+--------+
|  neg|  neu|  pos|compound|
+-----+-----+-----+--------+
|  0.0| 0.92| 0.08|  0.1027|
|0.141|0.859|  0.0| -0.4215|
|  0.0|0.851|0.149|  0.5411|
|  0.0|0.821|0.179|  0.5859|
|  0.0|0.816|0.184|  0.6467|
+-----+-----+-----+--------+
only showing top 5 rows



In [20]:
from pyspark.sql.functions import monotonically_increasing_id

ddf1 = dfdata.withColumn("row_id", monotonically_increasing_id())
ddf2 = df2.withColumn("row_id", monotonically_increasing_id())
result = ddf1.join(ddf2, ddf1.row_id == ddf2.row_id) #.drop("row_id")

+----+---------------+-------------+--------------------+--------------------+--------------------+---------+-----------------------+-------------+--------------+----+---------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+--------------------+------------------+------+-----+-----+-----+--------+------+
| _c0|scraped_hashtag|scraped_order|          created_at|              id_str|                text|truncated|in_reply_to_screen_name|retweet_count|favorite_count|lang|    screen_name|           user_name|    user_description|user_verified|user_followers_count|            hashtags|symbols|         og_tweet_by|og_tweet_truncated|row_id|  neg|  neu|  pos|compound|row_id|
+----+---------------+-------------+--------------------+--------------------+--------------------+---------+-----------------------+-------------+--------------+----+---------------+--------------------+--------------------+-------------+-------------------

In [23]:
result.show(1)

+---+---------------+-------------+--------------------+--------------------+--------------------+---------+-----------------------+-------------+--------------+----+---------------+-------------+--------------------+-------------+--------------------+--------------------+-------+-----------+------------------+------+---+---+---+--------+------+
|_c0|scraped_hashtag|scraped_order|          created_at|              id_str|                text|truncated|in_reply_to_screen_name|retweet_count|favorite_count|lang|    screen_name|    user_name|    user_description|user_verified|user_followers_count|            hashtags|symbols|og_tweet_by|og_tweet_truncated|row_id|neg|neu|pos|compound|row_id|
+---+---------------+-------------+--------------------+--------------------+--------------------+---------+-----------------------+-------------+--------------+----+---------------+-------------+--------------------+-------------+--------------------+--------------------+-------+-----------+-----------

In [26]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
# label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf]) #, label_stringIdx])

pipelineFit = pipeline.fit(result)
train_df = pipelineFit.transform(result)
# val_df = pipelineFit.transform(val_set)
# train_df.show(5)

In [27]:
train_df.show(1)

+---+---------------+-------------+--------------------+--------------------+--------------------+---------+-----------------------+-------------+--------------+----+---------------+-------------+--------------------+-------------+--------------------+--------------------+-------+-----------+------------------+------+---+---+---+--------+------+--------------------+--------------------+--------------------+
|_c0|scraped_hashtag|scraped_order|          created_at|              id_str|                text|truncated|in_reply_to_screen_name|retweet_count|favorite_count|lang|    screen_name|    user_name|    user_description|user_verified|user_followers_count|            hashtags|symbols|og_tweet_by|og_tweet_truncated|row_id|neg|neu|pos|compound|row_id|               words|                  tf|            features|
+---+---------------+-------------+--------------------+--------------------+--------------------+---------+-----------------------+-------------+--------------+----+------------

In [25]:
import pyspark.sql.functions as F
from pyspark.sql.types import  ArrayType, StringType
import re
    
#Add a dummy column to groupBy & in a single line
dfg = train_df.withColumn("dummy_col", 1)
dfg = train_df.withColumn("array_col", F.split("text", " "))
#Collect_set will return you an array without duplicates
df_grp = dfg.groupBy("dummy_col").agg(F.collect_set("array_col").alias("array_col"))
#explode to transpoe the column
df_grp = df_grp.withColumn("explode_col", F.explode("array_col"))
df_grp = df_grp.withColumn("explode_col", F.explode("explode_col"))
#Distince to remove the duplicates
df_grp = df_grp.select("explode_col").distinct()
#another dummy column to create the row number
df_grp = df_grp.withColumn("dummy_col", F.lit("A"))
_w = W.partitionBy("dummy_col").orderBy("dummy_col")
df_grp = df_grp.withColumn("rnk", F.row_number().over(_w))
df_grp.show(truncate=False)

AssertionError: col should be Column

In [ ]:
# def equivalent_type(f):
#     """
#     Read Pandas datatype and convert to PySpark datatype. 
#     """
#     if f == 'datetime64[ns]': return TimestampType()
#     elif f == 'int64': return LongType()
#     elif f == 'int32': return IntegerType()
#     elif f == 'float64': return FloatType()
#     else: return StringType()

# def define_structure(string, format_type):
#     """
#     Automatically define the structure of 
#     each column from pandas dataframe.
#     Returns PySpark StructFeild object. 
#     """
#     try: typo = equivalent_type(format_type)
#     except: typo = StringType()
#     return StructField(string, typo)

# # Given pandas dataframe, it will return a spark's dataframe.
# def pandas_to_spark(pandas_df):
#     """
#     Accepts pandas dataframe and converts to
#     PySpark dataframe. 
#     """
#     columns = list(pandas_df.columns)
#     types = list(pandas_df.dtypes)
#     struct_list = []
#     for column, typo in zip(columns, types): 
#       struct_list.append(define_structure(column, typo))
#     p_schema = StructType(struct_list)
#     return sqlContext.createDataFrame(pandas_df, p_schema)


sparkdf = pandas_to_spark(dfALL)
sparkdf.take(3)

In [ ]:
len(dfALL.index)

spark.read.option("delimiter", "\t").csv(file)

In [ ]:
# df2 = sparkdf.repartition(2)

In [ ]:
sparkdf.rdd.getNumPartitions()

In [ ]:
sparkdf.columns

In [ ]:
df2.count()

In [ ]:
data = dfALL[["scraped_hashtag", "text", "user_description"]].drop_duplicates()
data.index.names = ["doc_id"]
data = data.set_index(['scraped_hashtag'], append=True)
data['fulltext'] = data.text + " " + data.user_description.fillna("")
data = data.dropna().copy()
data = data.drop(columns=["text", "user_description"])
data
#477,640 unique records by doc_id, scraped_hashtag, and fulltext

In [ ]:
# from nltk.tokenize import TweetTokenizer

# tknzr = TweetTokenizer()

# def word_tokenize(x):
#     r = pd.Series(tknzr.tokenize(x))
#     #pd.Series(nltk.pos_tag(tknzr.tokenize(x)))
#     return r

# OHCO = ["doc_id", "scraped_hashtag", "token_num"]
# df = data.fulltext.apply(lambda x: pd.Series(tknzr.tokenize(x))).stack().to_frame().rename(columns={0:'term_str'})
# # df.index.names = OHCO

# # df['pos'] = df.pos_tuple.apply(lambda x: x[1])
# # df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
# # df = df.drop('pos_tuple', 1)
# # df['term_str'] = df['token_str'].str.lower().str.replace(r'[\W_]', '')
# # #ADD STEMS
# # stemmer = nltk.stem.SnowballStemmer('english')
# # df['p_stem'] = df.term_str.apply(stemmer.stem)

# df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(data.fulltext)

In [ ]:
feats = vectorizer.get_feature_names()
print(len(feats))
#Unique words in the dataset

In [ ]:
import nltk
nltk.download('stopwords')

stops = list(nltk.corpus.stopwords.words('english'))
nostops = [x for x in feats if x not in stops]
len(nostops)
#unique words in the dataset which are not stopwords

In [ ]:
vectorizer2 = CountVectorizer()

rsh = data.reset_index()
rsh.scraped_hashtag = rsh.scraped_hashtag + " x"
searchtags = vectorizer2.fit_transform(rsh.scraped_hashtag)
sfeats = vectorizer2.get_feature_names()
sfeats = list(set([str(x).replace("#", "") for x in sfeats]))
len(sfeats)

In [ ]:
sfeats

In [ ]:
TOKEN = df.copy()

In [ ]:
base_on = "p_stem"
voc = TOKEN[base_on].value_counts()\
    .to_frame()\
    .reset_index()\
    .rename(columns={base_on:'n', 'index':base_on})\
    .sort_values(base_on)
voc.index.name = 'term_id'

#LABEL STOPWORDS
stops = list(nltk.corpus.stopwords.words('english')) + ["thing", "one"]
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=[base_on])
sw = sw.reset_index().set_index(base_on)
sw.columns = ['dummy']
sw.dummy = 1
voc['stopword'] = voc[base_on].map(sw.dummy)
voc['stopword'] = voc['stopword'].fillna(0).astype('int')

#LABEL FILLER WORDS
fill_words = pd.DataFrame(["uh", "um", "uhm", "just", "like", \
    "yeah", "right", "okay",  \
    "alright", "so", "mhmm", "really"], columns=[base_on])
fill_words = fill_words.reset_index().set_index(base_on)
fill_words.columns = ['dummy']
fill_words.dummy = 1
voc['fill_word'] = voc[base_on].map(fill_words.dummy)
voc['fill_word'] = voc['fill_word'].fillna(0).astype('int')

# # #ADD STEMS
# # stemmer = nltk.stem.SnowballStemmer('english')
# # voc['p_stem'] = voc.term_str.apply(stemmer.stem)

# # #ADD LEMMATIZED
# # wnl = nltk.stem.WordNetLemmatizer()
# # voc['p_lemm'] = voc.term_str.apply(wnl.lemmatize)

#ADD POS_Max
poscount = TOKEN.reset_index()[[base_on, "pos", "token_num"]].groupby([base_on, "pos"]).count().reset_index()
posmax = poscount.groupby(base_on).max().reset_index()
posmax.columns = [base_on, "pos_max", "pos_count"]
voc = voc[[base_on, "n", "stopword", "fill_word"]].groupby(["term_id", base_on]).mean().reset_index()
voc = voc.merge(posmax[[base_on, "pos_max"]], how='left', on=base_on)
# voc = voc.set_index(["term_id"])

#FIND ENTROPY
n_tokens = voc.n.sum()
voc['p'] = voc['n'] / n_tokens
voc['i'] = np.log2(1/voc['p'])
n_terms = voc.shape[0]
H = (voc.p * voc.i).sum()
Hmax = np.log2(n_terms)
R = 1 - (H/Hmax)
print("Entropy of this vocab: ", R)

voc = voc.set_index(base_on)
voc.head(20)

In [ ]:
!pip install -U seaborn

In [ ]:
hist = np.log10(dfALL.query("user_followers_count > 100").user_followers_count).hist(bins=35)

In [ ]:
hist = np.log10(dfALL.query("user_followers_count > 100").user_followers_count.drop_duplicates()).hist(bins=35)

In [ ]:
 dfALL[["user_followers_count"]].describe()